In [ ]:
# ResNet 기본 블록 정의

import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(BasicBlock, self).__init__()

        # 합성곱층 정의 Conv2d 레이어
        # 입력과 출력의 크기를 맞추기 위해서 커널이 3일 때 패딩 1, 스트라이드 1 필요. stride 기본값은 1
        self.c1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
        self.c2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)

        # 스킵 커넥션할 입력의 크기 유지 + 출력 채널 수 변경 위해서 커널 값 1을 적용한 층
        self.downsample = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        #배치 정규화층 정의
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU()

    # Conv → BN → ReLU → Conv → BN 구조
    def forward(self, x):
        # 스킵 커넥션할 x_
        x_ = x

        # x 필터 통과
        x = self.c1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.bn2(x)

        # 합성곱의 결과와 입력의 채널 수를 맞춤(채널 수 늘어남)
        x_ = self.downsample(x_)

        #합성곱층의 결과(특징 맵)와 저장해놨던 입력값을 더해줌
        x += x_
        x = self.relu(x)

        return x

In [ ]:
# ResNet 모델 정의하기

class ResNet(nn.Module):
    def __init__(self, num_classes=10): # 분류할 클래스의 수가 10
        super(ResNet, self).__init__()

        # 기본 블록
        self.b1 = BasicBlock(in_channels=3, out_channels=64)
        self.b2 = BasicBlock(in_channels=64, out_channels=128)
        self.b3 = BasicBlock(in_channels=128, out_channels=256)
        # 풀링을 최댓값이 아닌 평균값으로
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        # fully connected layer(MLP)
        # (생각해보기) 왜 4096일까?
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)
        self.fc2 = nn.Linear(in_features=2048, out_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        # 기본 블록과 풀링층을 통과 = 전체 특징 추출 파트
        # 블록 수 계속 늘려보기
        x = self.b1(x)
        x = self.pool(x)
        x = self.b2(x)
        x = self.pool(x)
        x = self.b3(x)
        x = self.pool(x)

        # 분류기의 입력으로 사용하기 위해 flatten
        x = torch.flatten(x, start_dim=1)

        # 분류기로 예측값 출력: 최종 출력 생성
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

In [ ]:
#데이터 전처리부

import tqdm

from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor
from torchvision.transforms import RandomHorizontalFlip, RandomCrop
from torchvision.transforms import Normalize
from torch.utils.data.dataloader import DataLoader

from torch.optim.adam import Adam

train_transforms = Compose([
   RandomCrop((32, 32), padding=4),  #주변에 4픽셀 추가한 후, 32*32 크기로 랜덤 크롭핑
   RandomHorizontalFlip(p=0.5),  #50% 확률로 y축으로 뒤집기
   ToTensor(),  #텐서로 변환(픽셀값 범위를 정규화함)
   #이미지 정규화(R, G, B 각 채널별)
   Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

test_transform = Compose([
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

In [ ]:
#학습 데이터와 평가 데이터 불러오기
#transform = transforms 사용하면서 데이터의 증강이 적용됨.
#실제 저장공간에서의 데이터 수가 늘어나나? no. 하지만 매 epoch에서 랜덤한 변화가 생기면서 다양한 모습의 데이터를 볼 수 있음.
training_data = CIFAR10(root="./", train=True, download=True, transform=train_transforms)

#test데이터는 평가용이므로, 원본 데이터로 측정해야 함.
test_data = CIFAR10(root="./", train=False, download=True, transform=test_transform)

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet(num_classes=10)
model.to(device)

In [ ]:
lr = 1e-4
optim = Adam(model.parameters(), lr=lr)

for epoch in range(1):#시간관계상 1, 작동 테스트 후 큰 값으로 변경 바람
   iterator = tqdm.tqdm(train_loader)
   for data, label in iterator:
       # 최적화를 위해 기울기를 초기화
       optim.zero_grad()

       # 모델의 예측값
       preds = model(data.to(device))

       # 손실 계산 및 역전파
       loss = nn.CrossEntropyLoss()(preds, label.to(device))
       loss.backward()
       optim.step()

       iterator.set_description(f"epoch:{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "ResNet.pth")